In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Reload all modules imported with %aimport
%load_ext autoreload
%autoreload 1

In [2]:
import pandas as pd
idx = pd.IndexSlice

import datetime as dt
from datetime import date
from datetime import timedelta
import dateutil.parser as dup

%aimport trans.data
%aimport trans.gtrans
%aimport trans.reg
%aimport trans.regpipe

from trans.data import GetData
gd = GetData()
from trans.gtrans import *
from trans.reg import Reg, RegAttr
from trans.regpipe import RegPipe



In [3]:
today = dt.datetime.combine( date.today(), dt.time.min)
today

start = dup.parse("01/01/2000")
start

datetime.datetime(2018, 3, 9, 0, 0)

datetime.datetime(2000, 1, 1, 0, 0)

In [4]:
gd = GetData()
univ = gd.existing()
univ.sort()

len(univ)

910

In [5]:
sectors =  { 
    "Consumer Discretionary": "XLY",
    "Consumer Staples": "XLP",
    "Energy": "XLE",
    "Financial": "XLF",
    "Health": "XLV",
    "Industrial": "XLI", 
    "Materials" : "XLB",
    "Real Estate": "XLRE",
    "Technology": "XLK", 
    "Telecom": "XTL",
    "Utilities": "XLU"
}
   

In [6]:
sector_tickers = list( sectors.values() )

In [28]:
changed_tickers = gd.get_data( sector_tickers, start, today )

Extend SPY from 02/20/2018 beginning on 02/21/2018 to 03/08/2018.
Already have up-to-date XLP.
Already have up-to-date XLE.
Already have up-to-date XLF.
Already have up-to-date XLV.
Already have up-to-date XLI.
Already have up-to-date XLB.
Already have up-to-date XLRE.
Already have up-to-date XLK.
Already have up-to-date XTL.
Already have up-to-date XLU.


In [29]:
len(sector_tickers)
len(changed_tickers)
list( set(sector_tickers) - set(changed_tickers))

12

2

['XLP', 'XLE', 'XLI', 'XLK', 'XTL', 'XLV', 'XLB', 'XLF', 'XLRE', 'XLU']

In [7]:
price_df = GetDataTransformer(sector_tickers, cal_ticker="SPY").fit_transform( pd.DataFrame())
price_df.shape

(4574, 72)

## Compute returns

In [30]:
type(price_df.index)

pandas.tseries.index.DatetimeIndex

In [39]:
pipe_pct   = make_pipeline(GenSelectAttrsTransformer(['Adj Close'], dropSingle=False),
                           # RestrictToCalendarColTransformer( ("Adj Close", "SPY") ),
                           pctTrans,
                           # DatetimeIndexTransformer("Dt"),
                           GenRenameAttrsTransformer(lambda col: "Pct", level=0)
                          )
pct_df = pipe_pct.fit_transform(price_df)
pct_df.columns
pct_df.tail()

transform: pandas version <= 0.20.


MultiIndex(levels=[['Pct'], ['SPY', 'XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLRE', 'XLU', 'XLV', 'XLY', 'XTL']],
           labels=[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]])

Pct                                                    \
                 SPY       XLB       XLE       XLF       XLI       XLK   
Dt                                                                       
2018-03-02  0.005155 -0.001017  0.004652  0.002467 -0.000266  0.009547   
2018-03-05  0.011558  0.013229  0.010605  0.014065  0.011979  0.010195   
2018-03-06  0.002535  0.011048  0.000591  0.004161  0.004472  0.002194   
2018-03-07 -0.000366 -0.003642 -0.007533 -0.001726 -0.000131  0.005546   
2018-03-08  0.004582  0.001329 -0.000298  0.003113  0.005238  0.004354   

                                                                        
                 XLP      XLRE       XLU       XLV       XLY       XTL  
Dt                                                                      
2018-03-02  0.008444 -0.003338 -0.002649  0.010145  0.000971  0.013085  
2018-03-05  0.008746  0.014066  0.020433  0.009087  0.011936  0.013490  
2018-03-06  0.000738  0.003963 -0.013416 -0.001422  0.006904  0.016992  
2018-03-07 -0.009032  0.004276 -0.007510  0.005102 -0.006762  0.008911  
2018-03-08  0.008929  0.007861  0.007362  0.006375  0.003164 -0.001449

In [40]:
pipe_pct   = make_pipeline(GenSelectAttrsTransformer(['Adj Close'], dropSingle=True), 
                           # RestrictToCalendarColTransformer( "SPY" ),
                           pctTrans,
                           # DatetimeIndexTransformer("Dt"),
                           # RestrictToNonNullTransformer("all"),
                           AddAttrTransformer('Pct')
                      )
pct_df = pipe_pct.fit_transform(price_df)
pct_df.shape
pct_df.tail()

(4574, 12)

Pct                                                    \
                 SPY       XLB       XLE       XLF       XLI       XLK   
Dt                                                                       
2018-03-02  0.005155 -0.001017  0.004652  0.002467 -0.000266  0.009547   
2018-03-05  0.011558  0.013229  0.010605  0.014065  0.011979  0.010195   
2018-03-06  0.002535  0.011048  0.000591  0.004161  0.004472  0.002194   
2018-03-07 -0.000366 -0.003642 -0.007533 -0.001726 -0.000131  0.005546   
2018-03-08  0.004582  0.001329 -0.000298  0.003113  0.005238  0.004354   

                                                                        
                 XLP      XLRE       XLU       XLV       XLY       XTL  
Dt                                                                      
2018-03-02  0.008444 -0.003338 -0.002649  0.010145  0.000971  0.013085  
2018-03-05  0.008746  0.014066  0.020433  0.009087  0.011936  0.013490  
2018-03-06  0.000738  0.003963 -0.013416 -0.001422  0.006904  0.016992  
2018-03-07 -0.009032  0.004276 -0.007510  0.005102 -0.006762  0.008911  
2018-03-08  0.008929  0.007861  0.007362  0.006375  0.003164 -0.001449

In [10]:
import dateutil.parser as dup
import dateutil.relativedelta as rd

regWindow = rd.relativedelta(months=+6)
regStep   = rd.relativedelta(weeks=+4)

regStart = dup.parse("01/01/2000")
regEnd   = dup.parse("12/29/2017")

## Compute the model: 
$Return_{sector ticker} = \beta_0 + \beta * Return_{SPY} + \epsilon$

In [11]:
rp = RegPipe( pct_df )
rp.indCols( [ idx["Pct", "SPY"] ] )
rp.regress( regStart, regEnd, regWindow, regStep)

IndCols: [('Pct', 'SPY')], depCol ('Pct', 'XLB'), cols [('Pct', 'SPY'), ('Pct', 'XLB')]
IndCols: [('Pct', 'SPY')], depCol ('Pct', 'XLE'), cols [('Pct', 'SPY'), ('Pct', 'XLE')]
IndCols: [('Pct', 'SPY')], depCol ('Pct', 'XLF'), cols [('Pct', 'SPY'), ('Pct', 'XLF')]
IndCols: [('Pct', 'SPY')], depCol ('Pct', 'XLI'), cols [('Pct', 'SPY'), ('Pct', 'XLI')]
IndCols: [('Pct', 'SPY')], depCol ('Pct', 'XLK'), cols [('Pct', 'SPY'), ('Pct', 'XLK')]
IndCols: [('Pct', 'SPY')], depCol ('Pct', 'XLP'), cols [('Pct', 'SPY'), ('Pct', 'XLP')]
IndCols: [('Pct', 'SPY')], depCol ('Pct', 'XLRE'), cols [('Pct', 'SPY'), ('Pct', 'XLRE')]
Fit: the following columns have naN:  ['XLRE']
Fit: the following columns have naN:  ['XLRE']
Fit: the following columns have naN:  ['XLRE']
Fit: the following columns have naN:  ['XLRE']
Fit: the following columns have naN:  ['XLRE']
Fit: the following columns have naN:  ['XLRE']
Fit: the following columns have naN:  ['XLRE']
Fit: the following columns have naN:  ['XLRE']
Fit: t

In [12]:
rp.beta_df.tail()

Beta 0                                                    \
                 XLB       XLE       XLF       XLI       XLK       XLP   
Dt                                                                       
2017-09-08  0.000040 -0.000762 -0.000633 -0.000061  0.000298 -0.000031   
2017-10-06  0.000046 -0.000526  0.000056  0.000160  0.000109 -0.000334   
2017-11-03  0.000196 -0.000063  0.000188 -0.000023  0.000206 -0.000434   
2017-12-01  0.000067  0.000230  0.000597 -0.000055 -0.000096 -0.000412   
2017-12-29  0.000156  0.000496  0.000067  0.000079  0.000099 -0.000015   

                                                      ...       Beta 1  \
                XLRE       XLU       XLV       XLY    ...          XLE   
Dt                                                    ...                
2017-09-08  0.000423  0.000777  0.000454 -0.000135    ...     0.707524   
2017-10-06 -0.000031  0.000314  0.000363 -0.000267    ...     0.615224   
2017-11-03  0.000118  0.000597  0.000090 -0.000510    ...     0.541990   
2017-12-01  0.000124  0.000347  0.000080 -0.000298    ...     0.542585   
2017-12-29 -0.000165  0.000093 -0.000410  0.000037    ...     0.643274   

                                                                        \
                 XLF       XLI       XLK       XLP      XLRE       XLU   
Dt                                                                       
2017-09-08  1.297794  1.096055  1.280018  0.478708  0.508654  0.131469   
2017-10-06  1.294375  1.033866  1.340575  0.497813  0.461484  0.136299   
2017-11-03  1.133993  0.995060  1.439256  0.473676  0.460484  0.158490   
2017-12-01  0.993585  1.043711  1.461687  0.556895  0.520425  0.135866   
2017-12-29  1.131471  1.034340  1.369211  0.483971  0.568980  0.175113   

                                          
                 XLV       XLY       XTL  
Dt                                        
2017-09-08  0.834555  1.003449  1.326025  
2017-10-06  0.826531  1.009214  1.366022  
2017-11-03  0.827925  1.047990  1.318488  
2017-12-01  0.840500  1.025294  1.149932  
2017-12-29  0.921692  0.945197  1.158524  

[5 rows x 22 columns]

## Compute residuals:
 - For residual, don't roll beta: the date of the beta is the last date of the regression window
 - Fill the beta backwards, so the in-sample beta is applied

In [15]:
rollAmount = 0
fillMethod = "bfill"

rp.attrib_setup(pct_df, rp.beta_df, rollAmount, fillMethod)

addConst


/home/ubuntu/Notebooks/trans/reg.py:260: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[colName] = val


In [17]:
rp.attrib()
rp.retAttr_df.loc[:"2017-12-29",:].tail()

Contrib from 1                                                    \
                      XLB       XLE       XLF       XLI       XLK       XLP   
Dt                                                                            
2017-12-22       0.000156  0.000496  0.000067  0.000079  0.000099 -0.000015   
2017-12-26       0.000156  0.000496  0.000067  0.000079  0.000099 -0.000015   
2017-12-27       0.000156  0.000496  0.000067  0.000079  0.000099 -0.000015   
2017-12-28       0.000156  0.000496  0.000067  0.000079  0.000099 -0.000015   
2017-12-29       0.000156  0.000496  0.000067  0.000079  0.000099 -0.000015   

                                                     ...        Error  \
                XLRE       XLU      XLV       XLY    ...          XLE   
Dt                                                   ...                
2017-12-22 -0.000165  0.000093 -0.00041  0.000037    ...     0.001761   
2017-12-26 -0.000165  0.000093 -0.00041  0.000037    ...     0.009027   
2017-12-27 -0.000165  0.000093 -0.00041  0.000037    ...    -0.004115   
2017-12-28 -0.000165  0.000093 -0.00041  0.000037    ...    -0.000299   
2017-12-29 -0.000165  0.000093 -0.00041  0.000037    ...    -0.000968   

                                                                        \
                 XLF       XLI       XLK       XLP      XLRE       XLU   
Dt                                                                       
2017-12-22 -0.002615  0.000457 -0.000360  0.001727  0.006834  0.001480   
2017-12-26 -0.001565  0.002219 -0.005123  0.002880  0.006090 -0.005794   
2017-12-27  0.000455  0.001803  0.000327 -0.000395  0.003878  0.004041   
2017-12-28  0.001534 -0.000225 -0.001202 -0.002560  0.004802  0.005085   
2017-12-29 -0.002916  0.001974 -0.000224  0.001664  0.003222  0.001137   

                                          
                 XLV       XLY       XTL  
Dt                                        
2017-12-22 -0.002594 -0.001706 -0.004870  
2017-12-26  0.001754  0.003316 -0.001946  
2017-12-27  0.001650 -0.002513  0.002188  
2017-12-28 -0.000041  0.000745 -0.000069  
2017-12-29 -0.002245 -0.002316 -0.004190  

[5 rows x 44 columns]